In [1]:
pwd

'/nfs/hpc/share/kannegaa/DL-Project'

In [2]:

from imagenetv2_pytorch import ImageNetV2Dataset
import clip
from dataset import VLImageDataset
import torch
from tqdm import tqdm
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'.')
import torch
import torch.nn as nn
from tqdm import tqdm
import argparse
from importlib import reload,import_module
import multiprocessing
import os
import time
from itertools import product
sys.path.append("./PTQ4ViT")
import PTQ4ViT.utils.datasets as datasets
import PTQ4ViT.utils.net_wrap as net_wrap
from PTQ4ViT.utils.quant_calib import QuantCalibrator, HessianQuantCalibrator
from PTQ4ViT.utils.models import get_net
from dataset import VLImageDataset, VLLoader
from torchvision.datasets import CocoCaptions,Flickr30k
import torch.utils.data as dutils
from typing import List
import itertools


In [3]:
import numpy as np
import torch
import clip
from tqdm.notebook import tqdm
from pkg_resources import packaging

print("Torch version:", torch.__version__)


Torch version: 2.0.1+cu117


In [4]:
model, preprocess = clip.load("ViT-B/32", device="cuda")


In [5]:
def test_classification(net,test_loader,max_iteration=None, description=None):
    pos=0
    tot=0
    i = 0
    max_iteration = len(test_loader) if max_iteration is None else max_iteration
    with torch.no_grad():
        q=tqdm(test_loader, desc=description)
        for inp,target in q:
            i+=1
            inp=inp.cuda()
            target=target.cuda()
            out=net(inp)
            pos_num=torch.sum(out.argmax(1)==target).item()
            pos+=pos_num
            tot+=inp.size(0)
            q.set_postfix({"acc":pos/tot})
            if i >= max_iteration:
                break
    print(pos/tot)
    return pos/tot

In [6]:
def init_config(config_name):
    """initialize the config. Use reload to make sure it's fresh one!"""
    _,_,files =  next(os.walk("./PTQ4ViT/configs"))
    if config_name+".py" in files:
        quant_cfg = import_module(f"configs.{config_name}")
    else:
        raise NotImplementedError(f"Invalid config name {config_name}")
    reload(quant_cfg)
    return quant_cfg

In [7]:
def experiment_basic(net='vit_base_patch16_384', config="PTQ4ViT"):
    """
    A basic testbench.
    """
    quant_cfg = init_config(config)
    net = model
    #print("before:",net.eval())
    wrapped_modules = net_wrap.wrap_modules_in_net(net,quant_cfg)
    #print("after:",net.eval())
    # g=datasets.ViTImageNetLoaderGenerator('imagenetv2-matched-frequency-format-val','imagenet',32,32,16,kwargs={"model":net})
    # test_loader=g.test_loader()
    # calib_loader=g.calib_loader(num=32)
    image_features, text_features, target = VLLoader(model, preprocess)
    g = VLImageDataset(image_features.cuda(), text_features.cuda(), target.cuda())
    calib_loader = torch.utils.data.DataLoader(g, batch_size=32)

    quant_calibrator = HessianQuantCalibrator(net,wrapped_modules,calib_loader,sequential=False,batch_size=4) # 16 is too big for ViT-L-16
    quant_calibrator.batching_quant_calib()
    
    # test_classification(net,test_loader)
    return model



In [8]:
model = experiment_basic()
torch.save(model, "PTQ4CLIP.pt")
print("Model saved")

Completed net wrap.
[2806 5028 6437 2193 9476 1759 7372 4786 8569 1329 6229 9498 5671 7419
 8130    4  932 1157 6233 4633 1891 9106 1574 4532 5642 2329 2514 2915
 8963 5004 8466  116]
prepare parallel calibration for ['visual.conv1', 'visual.transformer.resblocks.0.mlp.c_fc', 'visual.transformer.resblocks.0.mlp.c_proj', 'visual.transformer.resblocks.1.mlp.c_fc', 'visual.transformer.resblocks.1.mlp.c_proj', 'visual.transformer.resblocks.2.mlp.c_fc', 'visual.transformer.resblocks.2.mlp.c_proj', 'visual.transformer.resblocks.3.mlp.c_fc', 'visual.transformer.resblocks.3.mlp.c_proj', 'visual.transformer.resblocks.4.mlp.c_fc', 'visual.transformer.resblocks.4.mlp.c_proj', 'visual.transformer.resblocks.5.mlp.c_fc', 'visual.transformer.resblocks.5.mlp.c_proj', 'visual.transformer.resblocks.6.mlp.c_fc', 'visual.transformer.resblocks.6.mlp.c_proj', 'visual.transformer.resblocks.7.mlp.c_fc', 'visual.transformer.resblocks.7.mlp.c_proj', 'visual.transformer.resblocks.8.mlp.c_fc', 'visual.transformer

Hessian:   0%|          | 0/49 [00:00<?, ?it/s, visual.conv1]/nfs/stak/users/kannegaa/.local/lib/python3.10/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
Hessian: 100%|██████████| 49/49 [02:36<00:00,  3.20s/it, transformer.resblocks.11.mlp.c_proj]       


hessian calibration finished
Model saved


In [19]:
#Image captioning
import argparse
import ipdb
from tqdm import tqdm
import progressbar
import torch
import ipdb
import sys

import itertools
from model.ZeroCLIP import CLIPTextGenerator




def get_args():
    parser = argparse.ArgumentParser()
    
    parser.add_argument("--test_image_prefix_path",default="/data/quantizeVL/image_captioning/data/mscoco/test_images", type=str, help="the folder that stores all test images")
    parser.add_argument("--test_path", default="/data/quantizeVL/image_captioning/data/mscoco/mscoco_test.json", type=str)
    parser.add_argument("--save_path_prefix", default="./inference_result/mscoco/baselines/",type=str, help="save the result in which directory")
    parser.add_argument("--save_name", default="zerocap_result.json", type=str, help="the name of the saved file")

    parser.add_argument("--seed", type=int, default=0)
    parser.add_argument("--lm_model", type=str, default="cambridgeltl/magic_mscoco", help="gpt-2 or gpt-neo")
    parser.add_argument("--clip_checkpoints", type=str, default="./clip_checkpoints", help="path to CLIP")
    parser.add_argument("--target_seq_length", type=int, default=16)
    parser.add_argument("--cond_text", type=str, default="Image of a")
    parser.add_argument("--reset_context_delta", action="store_false",
                        help="Should we reset the context at each token gen")
    parser.add_argument("--num_iterations", type=int, default=5)
    parser.add_argument("--clip_loss_temperature", type=float, default=0.01)
    parser.add_argument("--clip_scale", type=float, default=1)
    parser.add_argument("--ce_scale", type=float, default=0.2)
    parser.add_argument("--stepsize", type=float, default=0.3)
    parser.add_argument("--grad_norm_factor", type=float, default=0.9)
    parser.add_argument("--fusion_factor", type=float, default=0.99)
    parser.add_argument("--repetition_penalty", type=float, default=1)
    parser.add_argument("--end_token", type=str, default=".", help="Token to end text")
    parser.add_argument("--end_factor", type=float, default=1.01, help="Factor to increase end_token")
    parser.add_argument("--forbidden_factor", type=float, default=20, help="Factor to decrease forbidden tokens")
    parser.add_argument("--beam_size", type=int, default=1)

    parser.add_argument("--multi_gpu", action="store_true")

    parser.add_argument('--run_type',
                        default='caption',
                        nargs='?',
                        choices=['caption', 'arithmetics'])

    parser.add_argument("--caption_img_path", type=str, default='example_images/captions/COCO_val2014_000000008775.jpg',
                        help="Path to image for captioning")

    parser.add_argument("--arithmetics_imgs", nargs="+",
                        default=['example_images/arithmetics/woman2.jpg',
                                 'example_images/arithmetics/king2.jpg',
                                 'example_images/arithmetics/man2.jpg'])
    parser.add_argument("--arithmetics_weights", nargs="+", default=[1, 1, -1])

    args = parser.parse_args('')

    return args

def run(args, text_generator, img_path):
    image_features = text_generator.get_img_feature([img_path], None)
    captions = text_generator.run(image_features, args.cond_text, beam_size=args.beam_size)

    encoded_captions = [text_generator.clip.encode_text(clip.tokenize(c).to(text_generator.device)) for c in captions]
    encoded_captions = [x / x.norm(dim=-1, keepdim=True) for x in encoded_captions]
    best_clip_idx = (torch.cat(encoded_captions) @ image_features.t()).squeeze().argmax().item()
    return captions


if __name__ == '__main__':
    if '-f' in sys.argv:
        sys.argv.remove('-f')
    if torch.cuda.is_available():
        print ('Cuda is available.')
    cuda_available = torch.cuda.is_available()
    args = get_args()
    device = torch.device('cuda')

    save_path_prefix = args.save_path_prefix
    import os
    if os.path.exists(save_path_prefix):
        pass
    else: # recursively construct directory
        os.makedirs(save_path_prefix, exist_ok=True)
    # parse save name
    save_name = args.save_name
    full_save_path = save_path_prefix + '/' + save_name
    print ('full save path is {}'.format(full_save_path))

    print ('Loading data...')
    import json
    with open(args.test_path) as f:
        item_list = json.load(f)
    print ('Data loaded.')
    print ('Number of test instances is {}'.format(len(item_list)))

    # reloading model for each bit combination
    #model, preprocess = clip.load("RN50", device=device)
    # ZeroCap generator
    text_generator = CLIPTextGenerator(model, preprocess, **vars(args))
    result_list = []
    invalid_num = 0
    print ('----------------------------------------------------------------')
    #test_num = len(item_list)
    test_num = 100
    print ('Number of inference instances is {}'.format(test_num))
    p = progressbar.ProgressBar(test_num)
    p.start()
    print(args)
    for p_idx in tqdm(range(test_num)):   
        p.update(p_idx)
        one_test_dict = item_list[p_idx]
        one_res_dict = {'split':one_test_dict['split'],'image_name':one_test_dict['image_name'],'captions':one_test_dict['captions']}
        image_full_path = args.test_image_prefix_path + '/' + one_test_dict['image_name']
        output_text = run(args, text_generator, img_path=image_full_path)
        one_res_dict['prediction'] = output_text[0]
        result_list.append(one_res_dict)
        """       
        try:
            output_text = run(args, text_generator, img_path=image_full_path)
            one_res_dict['prediction'] = output_text[0]
            result_list.append(one_res_dict)
        except Exception as error:
            print(f'[!] ERROR:', error)
            invalid_num += 1
            print ('invalid number is {}'.format(invalid_num))
            continue
        """   
    p.finish()
    print ('Inference completed!')

    import json
    with open(full_save_path, 'w') as outfile:
              json.dump(result_list, outfile, indent=4)

 


Cuda is available.
full save path is ./inference_result/mscoco/baselines//zerocap_result.json
Loading data...
Data loaded.
Number of test instances is 5000


----------------------------------------------------------------
Number of inference instances is 100
Namespace(test_image_prefix_path='/data/quantizeVL/image_captioning/data/mscoco/test_images', test_path='/data/quantizeVL/image_captioning/data/mscoco/mscoco_test.json', save_path_prefix='./inference_result/mscoco/baselines/', save_name='zerocap_result.json', seed=0, lm_model='cambridgeltl/magic_mscoco', clip_checkpoints='./clip_checkpoints', target_seq_length=16, cond_text='Image of a', reset_context_delta=True, num_iterations=5, clip_loss_temperature=0.01, clip_scale=1, ce_scale=0.2, stepsize=0.3, grad_norm_factor=0.9, fusion_factor=0.99, repetition_penalty=1, end_token='.', end_factor=1.01, forbidden_factor=20, beam_size=1, multi_gpu=False, run_type='caption', caption_img_path='example_images/captions/COCO_val2014_000000008775.jpg', arithmetics_imgs=['example_images/arithmetics/woman2.jpg', 'example_images/arithmetics/king2.jpg', 'example_images/arithmetics/man2.jpg'], arithmetics_w

  0%|          | 0/100 [00:00<?, ?it/s]

16/06/2023 14:19:57 | [' traveling %% -2.736934']
16/06/2023 14:19:59 | [' traveling motorcycle %% -3.0408874']
16/06/2023 14:20:02 | [' traveling motorcycle with %% -2.8362572']
16/06/2023 14:20:05 | [' traveling motorcycle with man %% -2.8442726']
16/06/2023 14:20:07 | [' traveling motorcycle with maning %% -2.80654']
16/06/2023 14:20:10 | [' traveling motorcycle with maning and %% -2.8542635']
16/06/2023 14:20:13 | [' traveling motorcycle with maning and working %% -2.9977832']
16/06/2023 14:20:15 | [' traveling motorcycle with maning and working man %% -3.0449505']
16/06/2023 14:20:18 | [' traveling motorcycle with maning and working maning %% -3.1298027']
16/06/2023 14:20:21 | [' traveling motorcycle with maning and working maning on %% -3.10503']
16/06/2023 14:20:23 | [' traveling motorcycle with maning and working maning on a %% -3.0631227']
16/06/2023 14:20:26 | [' traveling motorcycle with maning and working maning on a wet %% -3.103503']
16/06/2023 14:20:29 | [' traveling mot

16/06/2023 14:20:34 | [' birthday %% -1.2025428']
16/06/2023 14:20:37 | [' birthday cake %% -1.9321597']
16/06/2023 14:20:39 | [' birthday cake with %% -2.0711973']
16/06/2023 14:20:42 | [' birthday cake with a %% -2.2944963']
16/06/2023 14:20:45 | [' birthday cake with a candle %% -2.4917102']
16/06/2023 14:20:47 | [' birthday cake with a candle licking %% -2.3897064']
16/06/2023 14:20:50 | [' birthday cake with a candle licking candle %% -2.5604682']
16/06/2023 14:20:53 | [' birthday cake with a candle licking candlelit %% -2.6110973']
16/06/2023 14:20:55 | [' birthday cake with a candle licking candlelit face %% -2.6826887']
16/06/2023 14:20:58 | [' birthday cake with a candle licking candlelit face. %% -2.743651']


...

16/06/2023 15:10:41 | [' photo %% -3.0997722']
16/06/2023 15:10:44 | [' photo with %% -3.0192242']
16/06/2023 15:10:47 | [' photo with a %% -2.887233']
16/06/2023 15:10:49 | [' photo with a bed %% -2.7945952']
16/06/2023 15:10:52 | [' photo with a bed and %% -2.8007581']
16/06/2023 15:10:55 | [' photo with a bed and a %% -2.8858116']
16/06/2023 15:10:57 | [' photo with a bed and a canopy %% -2.9610105']
16/06/2023 15:11:00 | [' photo with a bed and a canopy on %% -2.9493508']
16/06/2023 15:11:03 | [' photo with a bed and a canopy on a %% -2.8630266']
16/06/2023 15:11:06 | [' photo with a bed and a canopy on a wall %% -2.9192166']
16/06/2023 15:11:08 | [' photo with a bed and a canopy on a wall. %% -2.7667015']


16/06/2023 15:11:11 | [' pizz %% -0.90354514']
16/06/2023 15:11:14 | [' pizzas %% -2.3700619']
16/06/2023 15:11:16 | [' pizzasita %% -2.700512']
16/06/2023 15:11:19 | [' pizzasita with %% -2.6291623']
16/06/2023 15:11:22 | [' pizzasita with a %% -2.7282128']
16/06/2023 15:11:24 | [' pizzasita with a variety %% -2.8552399']
16/06/2023 15:11:27 | [' pizzasita with a variety of %% -2.603294']
16/06/2023 15:11:30 | [' pizzasita with a variety of topp %% -2.6016634']
16/06/2023 15:11:32 | [' pizzasita with a variety of toppings %% -2.3885007']
16/06/2023 15:11:35 | [' pizzasita with a variety of toppings and %% -2.4256415']
16/06/2023 15:11:38 | [' pizzasita with a variety of toppings and a %% -2.4466755']
16/06/2023 15:11:40 | [' pizzasita with a variety of toppings and a white %% -2.6019144']
16/06/2023 15:11:43 | [' pizzasita with a variety of toppings and a white sauce %% -2.6678605']
16/06/2023 15:11:46 | [' pizzasita with a variety of toppings and a white sauce. %% -2.6165578']


16/06/2023 15:11:49 | [' tennis %% -1.2497982']
16/06/2023 15:11:51 | [' tennis pink %% -1.0625532']
16/06/2023 15:11:54 | [' tennis pink pair %% -1.7243451']
16/06/2023 15:11:57 | [' tennis pink pair on %% -1.974869']
16/06/2023 15:11:59 | [' tennis pink pair on a %% -2.0509667']
16/06/2023 15:12:02 | [' tennis pink pair on a white %% -2.324915']
16/06/2023 15:12:05 | [' tennis pink pair on a white surface %% -2.4981549']
16/06/2023 15:12:07 | [' tennis pink pair on a white surface. %% -2.3100157']


100%|██████████| 100/100 [52:13<00:00, 31.33s/it]


Inference completed!


In [21]:
pwd

'/nfs/hpc/share/kannegaa/DL-Project'

In [3]:
%run -i './evaluation/cocoeval.py'
#python ./evaluation/cocoeval.py --result_file_path ./inference_result/mscoco/baselines/zerocap_result.json

tokenization...


PTBTokenizer tokenized 6114 tokens at 79602.96 tokens per second.
PTBTokenizer tokenized 1232 tokens at 32359.02 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 1043, 'reflen': 983, 'guess': [1043, 943, 843, 743], 'correct': [476, 127, 24, 6]}
ratio: 1.0610376398768453
Bleu_1: 0.456
Bleu_2: 0.248
Bleu_3: 0.121
Bleu_4: 0.061
computing METEOR score...
METEOR: 0.145
computing Rouge score...
ROUGE_L: 0.307
computing CIDEr score...
CIDEr: 0.325
computing SPICE score...


Parsing reference captions
Parsing test captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.3 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.1 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.4

SPICE evaluation took: 8.264 s
SPICE: 0.082
Bleu_1: 0.456
Bleu_2: 0.248
Bleu_3: 0.121
Bleu_4: 0.061
METEOR: 0.145
ROUGE_L: 0.307
CIDEr: 0.325
SPICE: 0.082


In [ ]:
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

In [ ]:
imagenet_classes = ["tench", "goldfish", "great white shark", "tiger shark", "hammerhead shark", "electric ray", "stingray", "rooster", "hen", "ostrich", "brambling", "goldfinch", "house finch", "junco", "indigo bunting", "American robin", "bulbul", "jay", "magpie", "chickadee", "American dipper", "kite (bird of prey)", "bald eagle", "vulture", "great grey owl", "fire salamander", "smooth newt", "newt", "spotted salamander", "axolotl", "American bullfrog", "tree frog", "tailed frog", "loggerhead sea turtle", "leatherback sea turtle", "mud turtle", "terrapin", "box turtle", "banded gecko", "green iguana", "Carolina anole", "desert grassland whiptail lizard", "agama", "frilled-necked lizard", "alligator lizard", "Gila monster", "European green lizard", "chameleon", "Komodo dragon", "Nile crocodile", "American alligator", "triceratops", "worm snake", "ring-necked snake", "eastern hog-nosed snake", "smooth green snake", "kingsnake", "garter snake", "water snake", "vine snake", "night snake", "boa constrictor", "African rock python", "Indian cobra", "green mamba", "sea snake", "Saharan horned viper", "eastern diamondback rattlesnake", "sidewinder rattlesnake", "trilobite", "harvestman", "scorpion", "yellow garden spider", "barn spider", "European garden spider", "southern black widow", "tarantula", "wolf spider", "tick", "centipede", "black grouse", "ptarmigan", "ruffed grouse", "prairie grouse", "peafowl", "quail", "partridge", "african grey parrot", "macaw", "sulphur-crested cockatoo", "lorikeet", "coucal", "bee eater", "hornbill", "hummingbird", "jacamar", "toucan", "duck", "red-breasted merganser", "goose", "black swan", "tusker", "echidna", "platypus", "wallaby", "koala", "wombat", "jellyfish", "sea anemone", "brain coral", "flatworm", "nematode", "conch", "snail", "slug", "sea slug", "chiton", "chambered nautilus", "Dungeness crab", "rock crab", "fiddler crab", "red king crab", "American lobster", "spiny lobster", "crayfish", "hermit crab", "isopod", "white stork", "black stork", "spoonbill", "flamingo", "little blue heron", "great egret", "bittern bird", "crane bird", "limpkin", "common gallinule", "American coot", "bustard", "ruddy turnstone", "dunlin", "common redshank", "dowitcher", "oystercatcher", "pelican", "king penguin", "albatross", "grey whale", "killer whale", "dugong", "sea lion", "Chihuahua", "Japanese Chin", "Maltese", "Pekingese", "Shih Tzu", "King Charles Spaniel", "Papillon", "toy terrier", "Rhodesian Ridgeback", "Afghan Hound", "Basset Hound", "Beagle", "Bloodhound", "Bluetick Coonhound", "Black and Tan Coonhound", "Treeing Walker Coonhound", "English foxhound", "Redbone Coonhound", "borzoi", "Irish Wolfhound", "Italian Greyhound", "Whippet", "Ibizan Hound", "Norwegian Elkhound", "Otterhound", "Saluki", "Scottish Deerhound", "Weimaraner", "Staffordshire Bull Terrier", "American Staffordshire Terrier", "Bedlington Terrier", "Border Terrier", "Kerry Blue Terrier", "Irish Terrier", "Norfolk Terrier", "Norwich Terrier", "Yorkshire Terrier", "Wire Fox Terrier", "Lakeland Terrier", "Sealyham Terrier", "Airedale Terrier", "Cairn Terrier", "Australian Terrier", "Dandie Dinmont Terrier", "Boston Terrier", "Miniature Schnauzer", "Giant Schnauzer", "Standard Schnauzer", "Scottish Terrier", "Tibetan Terrier", "Australian Silky Terrier", "Soft-coated Wheaten Terrier", "West Highland White Terrier", "Lhasa Apso", "Flat-Coated Retriever", "Curly-coated Retriever", "Golden Retriever", "Labrador Retriever", "Chesapeake Bay Retriever", "German Shorthaired Pointer", "Vizsla", "English Setter", "Irish Setter", "Gordon Setter", "Brittany dog", "Clumber Spaniel", "English Springer Spaniel", "Welsh Springer Spaniel", "Cocker Spaniel", "Sussex Spaniel", "Irish Water Spaniel", "Kuvasz", "Schipperke", "Groenendael dog", "Malinois", "Briard", "Australian Kelpie", "Komondor", "Old English Sheepdog", "Shetland Sheepdog", "collie", "Border Collie", "Bouvier des Flandres dog", "Rottweiler", "German Shepherd Dog", "Dobermann", "Miniature Pinscher", "Greater Swiss Mountain Dog", "Bernese Mountain Dog", "Appenzeller Sennenhund", "Entlebucher Sennenhund", "Boxer", "Bullmastiff", "Tibetan Mastiff", "French Bulldog", "Great Dane", "St. Bernard", "husky", "Alaskan Malamute", "Siberian Husky", "Dalmatian", "Affenpinscher", "Basenji", "pug", "Leonberger", "Newfoundland dog", "Great Pyrenees dog", "Samoyed", "Pomeranian", "Chow Chow", "Keeshond", "brussels griffon", "Pembroke Welsh Corgi", "Cardigan Welsh Corgi", "Toy Poodle", "Miniature Poodle", "Standard Poodle", "Mexican hairless dog (xoloitzcuintli)", "grey wolf", "Alaskan tundra wolf", "red wolf or maned wolf", "coyote", "dingo", "dhole", "African wild dog", "hyena", "red fox", "kit fox", "Arctic fox", "grey fox", "tabby cat", "tiger cat", "Persian cat", "Siamese cat", "Egyptian Mau", "cougar", "lynx", "leopard", "snow leopard", "jaguar", "lion", "tiger", "cheetah", "brown bear", "American black bear", "polar bear", "sloth bear", "mongoose", "meerkat", "tiger beetle", "ladybug", "ground beetle", "longhorn beetle", "leaf beetle", "dung beetle", "rhinoceros beetle", "weevil", "fly", "bee", "ant", "grasshopper", "cricket insect", "stick insect", "cockroach", "praying mantis", "cicada", "leafhopper", "lacewing", "dragonfly", "damselfly", "red admiral butterfly", "ringlet butterfly", "monarch butterfly", "small white butterfly", "sulphur butterfly", "gossamer-winged butterfly", "starfish", "sea urchin", "sea cucumber", "cottontail rabbit", "hare", "Angora rabbit", "hamster", "porcupine", "fox squirrel", "marmot", "beaver", "guinea pig", "common sorrel horse", "zebra", "pig", "wild boar", "warthog", "hippopotamus", "ox", "water buffalo", "bison", "ram (adult male sheep)", "bighorn sheep", "Alpine ibex", "hartebeest", "impala (antelope)", "gazelle", "arabian camel", "llama", "weasel", "mink", "European polecat", "black-footed ferret", "otter", "skunk", "badger", "armadillo", "three-toed sloth", "orangutan", "gorilla", "chimpanzee", "gibbon", "siamang", "guenon", "patas monkey", "baboon", "macaque", "langur", "black-and-white colobus", "proboscis monkey", "marmoset", "white-headed capuchin", "howler monkey", "titi monkey", "Geoffroy's spider monkey", "common squirrel monkey", "ring-tailed lemur", "indri", "Asian elephant", "African bush elephant", "red panda", "giant panda", "snoek fish", "eel", "silver salmon", "rock beauty fish", "clownfish", "sturgeon", "gar fish", "lionfish", "pufferfish", "abacus", "abaya", "academic gown", "accordion", "acoustic guitar", "aircraft carrier", "airliner", "airship", "altar", "ambulance", "amphibious vehicle", "analog clock", "apiary", "apron", "trash can", "assault rifle", "backpack", "bakery", "balance beam", "balloon", "ballpoint pen", "Band-Aid", "banjo", "baluster / handrail", "barbell", "barber chair", "barbershop", "barn", "barometer", "barrel", "wheelbarrow", "baseball", "basketball", "bassinet", "bassoon", "swimming cap", "bath towel", "bathtub", "station wagon", "lighthouse", "beaker", "military hat (bearskin or shako)", "beer bottle", "beer glass", "bell tower", "baby bib", "tandem bicycle", "bikini", "ring binder", "binoculars", "birdhouse", "boathouse", "bobsleigh", "bolo tie", "poke bonnet", "bookcase", "bookstore", "bottle cap", "hunting bow", "bow tie", "brass memorial plaque", "bra", "breakwater", "breastplate", "broom", "bucket", "buckle", "bulletproof vest", "high-speed train", "butcher shop", "taxicab", "cauldron", "candle", "cannon", "canoe", "can opener", "cardigan", "car mirror", "carousel", "tool kit", "cardboard box / carton", "car wheel", "automated teller machine", "cassette", "cassette player", "castle", "catamaran", "CD player", "cello", "mobile phone", "chain", "chain-link fence", "chain mail", "chainsaw", "storage chest", "chiffonier", "bell or wind chime", "china cabinet", "Christmas stocking", "church", "movie theater", "cleaver", "cliff dwelling", "cloak", "clogs", "cocktail shaker", "coffee mug", "coffeemaker", "spiral or coil", "combination lock", "computer keyboard", "candy store", "container ship", "convertible", "corkscrew", "cornet", "cowboy boot", "cowboy hat", "cradle", "construction crane", "crash helmet", "crate", "infant bed", "Crock Pot", "croquet ball", "crutch", "cuirass", "dam", "desk", "desktop computer", "rotary dial telephone", "diaper", "digital clock", "digital watch", "dining table", "dishcloth", "dishwasher", "disc brake", "dock", "dog sled", "dome", "doormat", "drilling rig", "drum", "drumstick", "dumbbell", "Dutch oven", "electric fan", "electric guitar", "electric locomotive", "entertainment center", "envelope", "espresso machine", "face powder", "feather boa", "filing cabinet", "fireboat", "fire truck", "fire screen", "flagpole", "flute", "folding chair", "football helmet", "forklift", "fountain", "fountain pen", "four-poster bed", "freight car", "French horn", "frying pan", "fur coat", "garbage truck", "gas mask or respirator", "gas pump", "goblet", "go-kart", "golf ball", "golf cart", "gondola", "gong", "gown", "grand piano", "greenhouse", "radiator grille", "grocery store", "guillotine", "hair clip", "hair spray", "half-track", "hammer", "hamper", "hair dryer", "hand-held computer", "handkerchief", "hard disk drive", "harmonica", "harp", "combine harvester", "hatchet", "holster", "home theater", "honeycomb", "hook", "hoop skirt", "gymnastic horizontal bar", "horse-drawn vehicle", "hourglass", "iPod", "clothes iron", "carved pumpkin", "jeans", "jeep", "T-shirt", "jigsaw puzzle", "rickshaw", "joystick", "kimono", "knee pad", "knot", "lab coat", "ladle", "lampshade", "laptop computer", "lawn mower", "lens cap", "letter opener", "library", "lifeboat", "lighter", "limousine", "ocean liner", "lipstick", "slip-on shoe", "lotion", "music speaker", "loupe magnifying glass", "sawmill", "magnetic compass", "messenger bag", "mailbox", "tights", "one-piece bathing suit", "manhole cover", "maraca", "marimba", "mask", "matchstick", "maypole", "maze", "measuring cup", "medicine cabinet", "megalith", "microphone", "microwave oven", "military uniform", "milk can", "minibus", "miniskirt", "minivan", "missile", "mitten", "mixing bowl", "mobile home", "ford model t", "modem", "monastery", "monitor", "moped", "mortar and pestle", "graduation cap", "mosque", "mosquito net", "vespa", "mountain bike", "tent", "computer mouse", "mousetrap", "moving van", "muzzle", "metal nail", "neck brace", "necklace", "baby pacifier", "notebook computer", "obelisk", "oboe", "ocarina", "odometer", "oil filter", "pipe organ", "oscilloscope", "overskirt", "bullock cart", "oxygen mask", "product packet / packaging", "paddle", "paddle wheel", "padlock", "paintbrush", "pajamas", "palace", "pan flute", "paper towel", "parachute", "parallel bars", "park bench", "parking meter", "railroad car", "patio", "payphone", "pedestal", "pencil case", "pencil sharpener", "perfume", "Petri dish", "photocopier", "plectrum", "Pickelhaube", "picket fence", "pickup truck", "pier", "piggy bank", "pill bottle", "pillow", "ping-pong ball", "pinwheel", "pirate ship", "drink pitcher", "block plane", "planetarium", "plastic bag", "plate rack", "farm plow", "plunger", "Polaroid camera", "pole", "police van", "poncho", "pool table", "soda bottle", "plant pot", "potter's wheel", "power drill", "prayer rug", "printer", "prison", "missile", "projector", "hockey puck", "punching bag", "purse", "quill", "quilt", "race car", "racket", "radiator", "radio", "radio telescope", "rain barrel", "recreational vehicle", "fishing casting reel", "reflex camera", "refrigerator", "remote control", "restaurant", "revolver", "rifle", "rocking chair", "rotisserie", "eraser", "rugby ball", "ruler measuring stick", "sneaker", "safe", "safety pin", "salt shaker", "sandal", "sarong", "saxophone", "scabbard", "weighing scale", "school bus", "schooner", "scoreboard", "CRT monitor", "screw", "screwdriver", "seat belt", "sewing machine", "shield", "shoe store", "shoji screen / room divider", "shopping basket", "shopping cart", "shovel", "shower cap", "shower curtain", "ski", "balaclava ski mask", "sleeping bag", "slide rule", "sliding door", "slot machine", "snorkel", "snowmobile", "snowplow", "soap dispenser", "soccer ball", "sock", "solar thermal collector", "sombrero", "soup bowl", "keyboard space bar", "space heater", "space shuttle", "spatula", "motorboat", "spider web", "spindle", "sports car", "spotlight", "stage", "steam locomotive", "through arch bridge", "steel drum", "stethoscope", "scarf", "stone wall", "stopwatch", "stove", "strainer", "tram", "stretcher", "couch", "stupa", "submarine", "suit", "sundial", "sunglasses", "sunglasses", "sunscreen", "suspension bridge", "mop", "sweatshirt", "swim trunks / shorts", "swing", "electrical switch", "syringe", "table lamp", "tank", "tape player", "teapot", "teddy bear", "television", "tennis ball", "thatched roof", "front curtain", "thimble", "threshing machine", "throne", "tile roof", "toaster", "tobacco shop", "toilet seat", "torch", "totem pole", "tow truck", "toy store", "tractor", "semi-trailer truck", "tray", "trench coat", "tricycle", "trimaran", "tripod", "triumphal arch", "trolleybus", "trombone", "hot tub", "turnstile", "typewriter keyboard", "umbrella", "unicycle", "upright piano", "vacuum cleaner", "vase", "vaulted or arched ceiling", "velvet fabric", "vending machine", "vestment", "viaduct", "violin", "volleyball", "waffle iron", "wall clock", "wallet", "wardrobe", "military aircraft", "sink", "washing machine", "water bottle", "water jug", "water tower", "whiskey jug", "whistle", "hair wig", "window screen", "window shade", "Windsor tie", "wine bottle", "airplane wing", "wok", "wooden spoon", "wool", "split-rail fence", "shipwreck", "sailboat", "yurt", "website", "comic book", "crossword", "traffic or street sign", "traffic light", "dust jacket", "menu", "plate", "guacamole", "consomme", "hot pot", "trifle", "ice cream", "popsicle", "baguette", "bagel", "pretzel", "cheeseburger", "hot dog", "mashed potatoes", "cabbage", "broccoli", "cauliflower", "zucchini", "spaghetti squash", "acorn squash", "butternut squash", "cucumber", "artichoke", "bell pepper", "cardoon", "mushroom", "Granny Smith apple", "strawberry", "orange", "lemon", "fig", "pineapple", "banana", "jackfruit", "cherimoya (custard apple)", "pomegranate", "hay", "carbonara", "chocolate syrup", "dough", "meatloaf", "pizza", "pot pie", "burrito", "red wine", "espresso", "tea cup", "eggnog", "mountain", "bubble", "cliff", "coral reef", "geyser", "lakeshore", "promontory", "sandbar", "beach", "valley", "volcano", "baseball player", "bridegroom", "scuba diver", "rapeseed", "daisy", "yellow lady's slipper", "corn", "acorn", "rose hip", "horse chestnut seed", "coral fungus", "agaric", "gyromitra", "stinkhorn mushroom", "earth star fungus", "hen of the woods mushroom", "bolete", "corn cob", "toilet paper"]

In [ ]:
imagenet_templates = [
    'a bad photo of a {}.',
    'a photo of many {}.',
    'a sculpture of a {}.',
    'a photo of the hard to see {}.',
    'a low resolution photo of the {}.',
    'a rendering of a {}.',
    'graffiti of a {}.',
    'a bad photo of the {}.',
    'a cropped photo of the {}.',
    'a tattoo of a {}.',
    'the embroidered {}.',
    'a photo of a hard to see {}.',
    'a bright photo of a {}.',
    'a photo of a clean {}.',
    'a photo of a dirty {}.',
    'a dark photo of the {}.',
    'a drawing of a {}.',
    'a photo of my {}.',
    'the plastic {}.',
    'a photo of the cool {}.',
    'a close-up photo of a {}.',
    'a black and white photo of the {}.',
    'a painting of the {}.',
    'a painting of a {}.',
    'a pixelated photo of the {}.',
    'a sculpture of the {}.',
    'a bright photo of the {}.',
    'a cropped photo of a {}.',
    'a plastic {}.',
    'a photo of the dirty {}.',
    'a jpeg corrupted photo of a {}.',
    'a blurry photo of the {}.',
    'a photo of the {}.',
    'a good photo of the {}.',
    'a rendering of the {}.',
    'a {} in a video game.',
    'a photo of one {}.',
    'a doodle of a {}.',
    'a close-up photo of the {}.',
    'a photo of a {}.',
    'the origami {}.',
    'the {} in a video game.',
    'a sketch of a {}.',
    'a doodle of the {}.',
    'a origami {}.',
    'a low resolution photo of a {}.',
    'the toy {}.',
    'a rendition of the {}.',
    'a photo of the clean {}.',
    'a photo of a large {}.',
    'a rendition of a {}.',
    'a photo of a nice {}.',
    'a photo of a weird {}.',
    'a blurry photo of a {}.',
    'a cartoon {}.',
    'art of a {}.',
    'a sketch of the {}.',
    'a embroidered {}.',
    'a pixelated photo of a {}.',
    'itap of the {}.',
    'a jpeg corrupted photo of the {}.',
    'a good photo of a {}.',
    'a plushie {}.',
    'a photo of the nice {}.',
    'a photo of the small {}.',
    'a photo of the weird {}.',
    'the cartoon {}.',
    'art of the {}.',
    'a drawing of the {}.',
    'a photo of the large {}.',
    'a black and white photo of a {}.',
    'the plushie {}.',
    'a dark photo of a {}.',
    'itap of a {}.',
    'graffiti of the {}.',
    'a toy {}.',
    'itap of my {}.',
    'a photo of a cool {}.',
    'a photo of a small {}.',
    'a tattoo of the {}.',
]

print(f"{len(imagenet_classes)} classes, {len(imagenet_templates)} templates")

In [ ]:
images = ImageNetV2Dataset(transform=preprocess)
loader = torch.utils.data.DataLoader(images, batch_size=32, num_workers=2)

In [ ]:
def zeroshot_classifier(classnames, templates):
    with torch.no_grad():
        zeroshot_weights = []
        for classname in tqdm(classnames):
            texts = [template.format(classname) for template in templates] #format with class
            texts = clip.tokenize(texts).cuda() #tokenize
            class_embeddings = model.encode_text(texts) #embed with text encoder
            class_embeddings /= class_embeddings.norm(dim=-1, keepdim=True)
            class_embedding = class_embeddings.mean(dim=0)
            class_embedding /= class_embedding.norm()
            zeroshot_weights.append(class_embedding)
        zeroshot_weights = torch.stack(zeroshot_weights, dim=1).cuda()
    return zeroshot_weights


zeroshot_weights = zeroshot_classifier(imagenet_classes, imagenet_templates)

In [ ]:
def accuracy(output, target, topk=(1,)):
    pred = output.topk(max(topk), 1, True, True)[1].t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))
    return [float(correct[:k].reshape(-1).float().sum(0, keepdim=True).cpu().numpy()) for k in topk]

In [ ]:
with torch.no_grad():
    top1, top5, n = 0., 0., 0.
    for i, (images, target) in enumerate(tqdm(loader)):
        images = images.cuda()
        target = target.cuda()
        
        # predict
        image_features = model.encode_image(images)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        logits = 100. * image_features @ zeroshot_weights

        # measure accuracy
        acc1, acc5 = accuracy(logits, target, topk=(1, 5))
        top1 += acc1
        top5 += acc5
        n += images.size(0)

top1 = (top1 / n) * 100
top5 = (top5 / n) * 100 

print(f"Top-1 accuracy: {top1:.2f}")
print(f"Top-5 accuracy: {top5:.2f}")

In [ ]:
device='cuda'

In [ ]:
# Change these to path of local COCO dataset:
coco_root = "/data/quantizeVL/retrieval/data/mscoco/val2017"
coco_ann_file = "/data/quantizeVL/retrieval/data/mscoco/annotations/captions_val2017.json"
dataset = CocoCaptions(
   root=coco_root,
   annFile=coco_ann_file,
   transform=preprocess,
   # Note: almost all images have 5 captions, but 12/5000 have 6, and 1/5000 has 7 - I ignore these few extra captions.
   target_transform=lambda texts: clip.tokenize(texts[:5])
)

k_vals=[1, 5]

In [ ]:
def encode_dataset(clip, dataset: dutils.Dataset, batch_size = 16):
    with torch.no_grad():
        # image_to_text_map[i] gives the corresponding text indices for the ith image
        #  (as there are multiple pieces of text for each image)
        image_to_text_map = []

        # text_to_image_map[i] gives the corresponding image index for the ith text
        text_to_image_map = []

        dataloader = dutils.DataLoader(dataset, batch_size=batch_size, shuffle=False)

        image_encodings = []
        text_encodings = []

        text_index = 0
        image_index = 0

        for images, text in dataloader:
            images = images.to(device)
            text = text.to(device)

            # text has shape B x 5 x 77
            batch_size, captions_per_image, _ = text.shape

            # Update text_to_image_map and image_to_text_map for this batch
            for i in range(batch_size):
                # the next image corresponds to text captions [text_index ... text_index + captions_per_image - 1]
                text_indices = list(range(text_index, text_index + captions_per_image))
                image_to_text_map.append(text_indices)
                text_index += captions_per_image

                # Each of the next captions_per_image text captions correspond to the same image
                text_to_image_map += [image_index] * captions_per_image
                image_index += 1
                
            # B x 5 x 77 -> (B*5) x 77
            text = torch.flatten(text, start_dim=0, end_dim=1)

            image_encodings.append(clip.encode_image(images))
            text_encodings.append(clip.encode_text(text))

        image_encodings = torch.cat(image_encodings)
        text_encodings = torch.cat(text_encodings)
        text_to_image_map = torch.LongTensor(text_to_image_map).to(device)
        image_to_text_map = torch.LongTensor(image_to_text_map).to(device)

        # Normalise encodings
        image_encodings = image_encodings / image_encodings.norm(dim=-1, keepdim=True)
        text_encodings = text_encodings / text_encodings.norm(dim=-1, keepdim=True)

        return image_encodings, text_encodings, text_to_image_map, image_to_text_map


def recall_at_k(clip, dataset: dutils.Dataset, k_vals: List[int], batch_size: int):
    print("Encoding all data...")
    image_encodings, text_encodings, text_to_image_map, image_to_text_map = encode_dataset(clip, dataset, batch_size=batch_size)

    num_text = text_encodings.shape[0]
    num_im = image_encodings.shape[0]
    captions_per_image = image_to_text_map.shape[1]

    # text-to-image recall
    print("Text-to-image recall...")

    dist_matrix = text_encodings @ image_encodings.T  # dist_matrix[i] gives logits for ith text

    # Note: this matrix is pretty big (5000 x 25000 with dtype float16 = 250MB)
    #  torch.argsort runs out of memory for me (6GB VRAM) so I move to CPU for sorting
    dist_matrix = dist_matrix.cpu()

    # Sort in descending order; first is the biggest logit
    inds = torch.argsort(dist_matrix, dim=1, descending=True)
    inds = inds.to(device)

    text_to_image_recall = []

    for k in k_vals:
        # Extract top k indices only
        topk = inds[:, :k]

        # Correct iff one of the top_k values equals the correct image (as given by text_to_image_map)
        correct = torch.eq(topk, text_to_image_map.unsqueeze(-1)).any(dim=1)

        num_correct = correct.sum().item()
        text_to_image_recall.append(num_correct / num_text)


    # image-to-text recall
    print("Image-to-text recall...")
    dist_matrix = dist_matrix.T  # dist_matrix[i] gives logits for the ith image

    # Sort in descending order; first is the biggest logit
    inds = torch.argsort(dist_matrix, dim=1, descending=True)
    inds = inds.to(device)

    image_to_text_recall = []

    for k in k_vals:
        # Extract top k indices only
        topk = inds[:, :k]

        correct = torch.zeros((num_im,), dtype=torch.bool).cuda()

        #  For each image, check whether one of the 5 relevant captions was retrieved
        # Check if image matches its ith caption (for i=0..4)
        for i in range(captions_per_image):
            contains_index = torch.eq(topk, image_to_text_map[:, i].unsqueeze(-1)).any(dim=1)
            correct = torch.logical_or(correct, contains_index)

        num_correct = correct.sum().item()
        image_to_text_recall.append(num_correct / num_im)#

    print("Done.")
    return text_to_image_recall, image_to_text_recall

In [ ]:

 t2i, i2t = recall_at_k(model, dataset, k_vals=k_vals, batch_size=16)
 
 print("Text-to-image Recall@K")
 for k, x in zip(k_vals, t2i):
     print(f" R@{k}: {100*x:.2f}%")

 print("Image-to-text Recall@K")
 for k, x in zip(k_vals, i2t):
     print(f" R@{k}: {100*x:.2f}%")
 print(" ")
                                      

In [ ]:
# Image Captioning